In [2]:
import scipy.io
import numpy as np
from random import shuffle
import random
import spectral
import scipy.ndimage
from skimage.transform import rotate
import os
import patch_size
%matplotlib inline
from pprint import pprint

In [3]:
input_mat = np.load('input_m.npy')
target_mat = np.load('target_m.npy')
input_m = np.load('Input.npy')
target_m = np.load('GT.npy')
target_mat1 = np.load('GroundTruth_CMF2.npy')

pprint(target_mat.shape)
pprint(target_mat1.shape)

for i in range(0,target_mat1.shape[0]):
    for j in range(0,target_mat1.shape[1]):
        if(target_mat1[i][j]==0):
            target_mat1[i][j]=2
pprint(target_mat1)

(145, 145)
(309, 512)
array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.]], dtype=float32)


In [4]:
HEIGHT = input_mat.shape[0]
WIDTH = input_mat.shape[1]
BAND = input_mat.shape[2]
output_class = 2
PATCH_SIZE = patch_size.patch_size
TRAIN_PATCH,TRAIN_LABELS,TEST_PATCH,TEST_LABELS = [],[],[],[]
CLASSES = [] 
COUNT = 200 #Number of patches of each class
OUTPUT_CLASSES = 16
TEST_FRAC = 0.25 #Fraction of data to be used for testing

In [5]:
PATCH_SIZE

1

In [6]:
input_mat = input_mat.astype(float)
input_mat -= np.min(input_mat)
input_mat /= np.max(input_mat)

In [7]:
MEAN_ARRAY = np.ndarray(shape=(BAND,),dtype=float)
for i in range(BAND):
    MEAN_ARRAY[i] = np.mean(input_mat[:,:,i])

In [8]:
def Patch(height_index,width_index):
    transpose_array = np.transpose(input_mat,(2,0,1))
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = transpose_array[:, height_slice, width_slice]
    mean_normalized_patch = []
    for i in range(patch.shape[0]):
        mean_normalized_patch.append(patch[i] - MEAN_ARRAY[i]) 
    
    return np.array(mean_normalized_patch)

In [9]:
for i in range(OUTPUT_CLASSES):
    CLASSES.append([])
for i in range(HEIGHT - PATCH_SIZE + 1):
    for j in range(WIDTH - PATCH_SIZE + 1):
        curr_inp = Patch(i,j)
        curr_tar = target_mat[i + int((PATCH_SIZE - 1)/2), j + int((PATCH_SIZE - 1)/2)]
        if(curr_tar!=0): #Ignore patches with unknown landcover type for the central pixel
            CLASSES[curr_tar-1].append(curr_inp)

In [10]:
for c  in CLASSES:
    print(len(c))

46
1428
830
237
483
730
28
478
20
972
2455
593
205
1265
386
93


In [11]:
for c in range(OUTPUT_CLASSES): #for each class
    class_population = len(CLASSES[c])
    test_split_size = int(class_population*TEST_FRAC)
        
    patches_of_current_class = CLASSES[c]
    shuffle(patches_of_current_class)
    
    #Make training and test splits
    TRAIN_PATCH.append(patches_of_current_class[:-test_split_size])
        
    TEST_PATCH.extend(patches_of_current_class[-test_split_size:])
    TEST_LABELS.extend(np.full(test_split_size, c, dtype=int))

In [26]:
#for c in TRAIN_PATCH:
#    print(len(c))
#print(len(TRAIN_PATCH))

In [27]:
for k in range(OUTPUT_CLASSES):
    if(len(TRAIN_PATCH[k])<COUNT):
        tmp = TRAIN_PATCH[k]
        for j in range(int(COUNT/len(TRAIN_PATCH[k]))):
            shuffle(TRAIN_PATCH[k])
            TRAIN_PATCH[k] = TRAIN_PATCH[k] + tmp
    shuffle(TRAIN_PATCH[k])
    TRAIN_PATCH[k] = TRAIN_PATCH[k][:COUNT]

In [29]:
#for c in TRAIN_PATCH:
 #   print(len(c))


In [30]:
TRAIN_PATCH = np.asarray(TRAIN_PATCH)

#print(TRAIN_PATCH)

In [31]:
TRAIN_PATCH = TRAIN_PATCH.reshape((-1,220,PATCH_SIZE,PATCH_SIZE))

In [32]:
TRAIN_LABELS = np.array([])
for l in range(OUTPUT_CLASSES):
    TRAIN_LABELS = np.append(TRAIN_LABELS, np.full(COUNT, l, dtype=int))

In [34]:
#print(len(TEST_PATCH))
#print(len(TRAIN_PATCH))

Save the patches in segments
=================================

1. Training data
----------------

In [35]:
for i in range(int(len(TRAIN_PATCH)/(COUNT*2))):
    train_dict = {}
    start = i * (COUNT*2)
    end = (i+1) * (COUNT*2)
    file_name = 'Train_'+str(PATCH_SIZE)+'_'+str(i+1)+'.mat'
    train_dict["train_patch"] = TRAIN_PATCH[start:end]
    train_dict["train_labels"] = TRAIN_LABELS[start:end]
    scipy.io.savemat(os.path.join(DATA_PATH, file_name),train_dict)


2. Test data
-------------

In [36]:
for i in range(int(len(TEST_PATCH)/(COUNT*2))):
    test_dict = {}
    start = i * (COUNT*2)
    end = (i+1) * (COUNT*2)
    file_name = 'Test_'+str(PATCH_SIZE)+'_'+str(i+1)+'.mat'
    test_dict["test_patch"] = TEST_PATCH[start:end]
    test_dict["test_labels"] = TEST_LABELS[start:end]
    scipy.io.savemat(os.path.join(DATA_PATH, file_name),test_dict)

In [37]:
len(TRAIN_PATCH)/(COUNT*2)


8.0